In [1]:
# GRU MODELS  

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense

# Load the time series data
data = pd.read_csv('NMC.csv')

# Ensure the data has the necessary columns: 'Open', 'High', 'Close', 'Volume', 'RSI', 'MA'
assert set(['Open', 'High', 'Close', 'Volume', 'RSI', 'MA']).issubset(data.columns), "Missing necessary columns in the data"

# Step 1: Labeling the data
def label_data(df, interval=4):
    labels = []
    for i in range(0, len(df) - interval + 1):
        if df['Open'][i] < df['Close'][i + interval - 1]:
            labels.append(1)
        else:
            labels.append(0)
    labels.extend([np.nan] * (interval - 1))
    return labels

data['Label'] = label_data(data)

# Drop rows with NaN labels
data = data.dropna(subset=['Label'])

# Step 2: Splitting the data
train_size = int(len(data) * 0.7)
val_size = int(len(data) * 0.2)
test_size = len(data) - train_size - val_size

train_data = data.iloc[:train_size]
val_data = data.iloc[train_size:train_size + val_size]
test_data = data.iloc[train_size + val_size:]

# Step 3: Normalizing the data (without data leakage)
scaler = StandardScaler()
features = ['Open', 'High', 'Close', 'Volume', 'RSI', 'MA']

train_data.loc[:, features] = scaler.fit_transform(train_data[features])
val_data.loc[:, features] = scaler.transform(val_data[features])
test_data.loc[:, features] = scaler.transform(test_data[features])

# Ensure there are no NaN values
assert not train_data[features].isnull().values.any(), "Train data contains NaN values"
assert not val_data[features].isnull().values.any(), "Validation data contains NaN values"
assert not test_data[features].isnull().values.any(), "Test data contains NaN values"

# Step 4: Creating sequences
def create_sequences(data, time_steps=4):
    X, y = [], []
    for i in range(len(data) - time_steps + 1):
        X.append(data.iloc[i:(i + time_steps), :].drop(columns=['Datetime', 'Label']).values)
        y.append(data.iloc[i + time_steps - 1]['Label'])
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_data)
X_val, y_val = create_sequences(val_data)
X_test, y_test = create_sequences(test_data)

# Ensure all data is of type float
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)
X_val = X_val.astype(np.float32)
y_val = y_val.astype(np.float32)
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.float32)

# Check the shapes and types of the data
print(f"X_train shape: {X_train.shape}, dtype: {X_train.dtype}")
print(f"y_train shape: {y_train.shape}, dtype: {y_train.dtype}")
print(f"X_val shape: {X_val.shape}, dtype: {X_val.dtype}")
print(f"y_val shape: {y_val.shape}, dtype: {y_val.dtype}")
print(f"X_test shape: {X_test.shape}, dtype: {X_test.dtype}")
print(f"y_test shape: {y_test.shape}, dtype: {y_test.dtype}")

# Step 5: Building the GRU model
model = Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1], X_train.shape[2])),
    GRU(128, return_sequences=True),
    GRU(128),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 6: Training the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

# Step 7: Evaluating the model
def evaluate_model(model, X, y):
    y_pred = model.predict(X).round()
    auroc = roc_auc_score(y, y_pred)
    auprc = average_precision_score(y, y_pred)
    accuracy = accuracy_score(y, y_pred)
    precision = precision_score(y, y_pred)
    recall = recall_score(y, y_pred)
    f1 = f1_score(y, y_pred)
    return auroc, auprc, accuracy, precision, recall, f1

# Evaluation on training-validation set
train_val_auroc, train_val_auprc, train_val_accuracy, train_val_precision, train_val_recall, train_val_f1 = evaluate_model(model, X_train, y_train)
val_auroc, val_auprc, val_accuracy, val_precision, val_recall, val_f1 = evaluate_model(model, X_val, y_val)

# Evaluation on test set
test_auroc, test_auprc, test_accuracy, test_precision, test_recall, test_f1 = evaluate_model(model, X_test, y_test)

print("Training-Validation Metrics:")
print(f"AUROC: {train_val_auroc}")
print(f"AUPRC: {train_val_auprc}")
print(f"Accuracy: {train_val_accuracy}")
print(f"Precision: {train_val_precision}")
print(f"Recall: {train_val_recall}")
print(f"F1 Score: {train_val_f1}")

print("\nValidation Metrics:")
print(f"AUROC: {val_auroc}")
print(f"AUPRC: {val_auprc}")
print(f"Accuracy: {val_accuracy}")
print(f"Precision: {val_precision}")
print(f"Recall: {val_recall}")
print(f"F1 Score: {val_f1}")

print("\nTest Metrics:")
print(f"AUROC: {test_auroc}")
print(f"AUPRC: {test_auprc}")
print(f"Accuracy: {test_accuracy}")
print(f"Precision: {test_precision}")
print(f"Recall: {test_recall}")
print(f"F1 Score: {test_f1}")

2024-06-14 05:11:20.908585: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-14 05:11:22.576474: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


X_train shape: (12244, 4, 6), dtype: float32
y_train shape: (12244,), dtype: float32
X_val shape: (3496, 4, 6), dtype: float32
y_val shape: (3496,), dtype: float32
X_test shape: (1747, 4, 6), dtype: float32
y_test shape: (1747,), dtype: float32
Epoch 1/50
383/383 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.5274 - loss: 0.6916 - val_accuracy: 0.5995 - val_loss: 0.6687
Epoch 2/50
383/383 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.5862 - loss: 0.6708 - val_accuracy: 0.5841 - val_loss: 0.6733
Epoch 3/50
383/383 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.5812 - loss: 0.6699 - val_accuracy: 0.5950 - val_loss: 0.6685
Epoch 4/50
383/383 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.5793 - loss: 0.6702 - val_accuracy: 0.5721 - val_loss: 0.6751
Epoch 5/50
383/383 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.5811 - loss: 0.6686 - val_accuracy: 0.6035 - val_loss: 0.6647
Epoch 6/50
383/383 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.5885 - loss: 0.6608 - val_accuracy: 0.6030 - val_

In [2]:
# LSTM 

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load the time series data
data = pd.read_csv('NMC.csv')

# Ensure the data has the necessary columns: 'Open', 'High', 'Close', 'Volume', 'RSI', 'MA'
assert set(['Open', 'High', 'Close', 'Volume', 'RSI', 'MA']).issubset(data.columns), "Missing necessary columns in the data"

# Step 1: Labeling the data
def label_data(df, interval=4):
    labels = []
    for i in range(0, len(df) - interval + 1):
        if df['Open'][i] < df['Close'][i + interval - 1]:
            labels.append(1)
        else:
            labels.append(0)
    labels.extend([np.nan] * (interval - 1))
    return labels

data['Label'] = label_data(data)

# Drop rows with NaN labels
data = data.dropna(subset=['Label'])

# Step 2: Splitting the data
train_size = int(len(data) * 0.7)
val_size = int(len(data) * 0.2)
test_size = len(data) - train_size - val_size

train_data = data.iloc[:train_size]
val_data = data.iloc[train_size:train_size + val_size]
test_data = data.iloc[train_size + val_size:]

# Step 3: Normalizing the data (without data leakage)
scaler = StandardScaler()
features = ['Open', 'High', 'Close', 'Volume', 'RSI', 'MA']

train_data.loc[:, features] = scaler.fit_transform(train_data[features])
val_data.loc[:, features] = scaler.transform(val_data[features])
test_data.loc[:, features] = scaler.transform(test_data[features])

# Ensure there are no NaN values
assert not train_data[features].isnull().values.any(), "Train data contains NaN values"
assert not val_data[features].isnull().values.any(), "Validation data contains NaN values"
assert not test_data[features].isnull().values.any(), "Test data contains NaN values"

# Step 4: Creating sequences
def create_sequences(data, time_steps=4):
    X, y = [], []
    for i in range(len(data) - time_steps + 1):
        X.append(data.iloc[i:(i + time_steps), :].drop(columns=['Datetime', 'Label']).values)
        y.append(data.iloc[i + time_steps - 1]['Label'])
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_data)
X_val, y_val = create_sequences(val_data)
X_test, y_test = create_sequences(test_data)

# Ensure all data is of type float
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)
X_val = X_val.astype(np.float32)
y_val = y_val.astype(np.float32)
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.float32)

# Check the shapes and types of the data
print(f"X_train shape: {X_train.shape}, dtype: {X_train.dtype}")
print(f"y_train shape: {y_train.shape}, dtype: {y_train.dtype}")
print(f"X_val shape: {X_val.shape}, dtype: {X_val.dtype}")
print(f"y_val shape: {y_val.shape}, dtype: {y_val.dtype}")
print(f"X_test shape: {X_test.shape}, dtype: {X_test.dtype}")
print(f"y_test shape: {y_test.shape}, dtype: {y_test.dtype}")

# Step 5: Building the LSTM model
model = Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(128, return_sequences=True),
    LSTM(128),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 6: Training the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

# Step 7: Evaluating the model
def evaluate_model(model, X, y):
    y_pred = model.predict(X).round()
    auroc = roc_auc_score(y, y_pred)
    auprc = average_precision_score(y, y_pred)
    accuracy = accuracy_score(y, y_pred)
    precision = precision_score(y, y_pred)
    recall = recall_score(y, y_pred)
    f1 = f1_score(y, y_pred)
    return auroc, auprc, accuracy, precision, recall, f1

# Evaluation on training-validation set
train_val_auroc, train_val_auprc, train_val_accuracy, train_val_precision, train_val_recall, train_val_f1 = evaluate_model(model, X_train, y_train)
val_auroc, val_auprc, val_accuracy, val_precision, val_recall, val_f1 = evaluate_model(model, X_val, y_val)

# Evaluation on test set
test_auroc, test_auprc, test_accuracy, test_precision, test_recall, test_f1 = evaluate_model(model, X_test, y_test)

print("Training-Validation Metrics:")
print(f"AUROC: {train_val_auroc}")
print(f"AUPRC: {train_val_auprc}")
print(f"Accuracy: {train_val_accuracy}")
print(f"Precision: {train_val_precision}")
print(f"Recall: {train_val_recall}")
print(f"F1 Score: {train_val_f1}")

print("\nValidation Metrics:")
print(f"AUROC: {val_auroc}")
print(f"AUPRC: {val_auprc}")
print(f"Accuracy: {val_accuracy}")
print(f"Precision: {val_precision}")
print(f"Recall: {val_recall}")
print(f"F1 Score: {val_f1}")

print("\nTest Metrics:")
print(f"AUROC: {test_auroc}")
print(f"AUPRC: {test_auprc}")
print(f"Accuracy: {test_accuracy}")
print(f"Precision: {test_precision}")
print(f"Recall: {test_recall}")
print(f"F1 Score: {test_f1}")

X_train shape: (12244, 4, 6), dtype: float32
y_train shape: (12244,), dtype: float32
X_val shape: (3496, 4, 6), dtype: float32
y_val shape: (3496,), dtype: float32
X_test shape: (1747, 4, 6), dtype: float32
y_test shape: (1747,), dtype: float32
Epoch 1/50
383/383 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.5289 - loss: 0.6922 - val_accuracy: 0.5429 - val_loss: 0.6872
Epoch 2/50
383/383 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.5662 - loss: 0.6796 - val_accuracy: 0.5901 - val_loss: 0.6705
Epoch 3/50
383/383 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.5755 - loss: 0.6721 - val_accuracy: 0.5941 - val_loss: 0.6676
Epoch 4/50
383/383 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.5854 - loss: 0.6688 - val_accuracy: 0.5932 - val_loss: 0.6694
Epoch 5/50
383/383 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.5818 - loss: 0.6707 - val_accuracy: 0.5890 - val_loss: 0.6686
Epoch 6/50
383/383 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.5907 - loss: 0.6643 - val_accuracy: 0.5941 - val

In [3]:
# CNN MODEL 
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# Load the time series data
data = pd.read_csv('NMC.csv')

# Ensure the data has the necessary columns: 'Open', 'High', 'Close', 'Volume', 'RSI', 'MA'
assert set(['Open', 'High', 'Close', 'Volume', 'RSI', 'MA']).issubset(data.columns), "Missing necessary columns in the data"

# Step 1: Labeling the data
def label_data(df, interval=4):
    labels = []
    for i in range(0, len(df) - interval + 1):
        if df['Open'][i] < df['Close'][i + interval - 1]:
            labels.append(1)
        else:
            labels.append(0)
    labels.extend([np.nan] * (interval - 1))
    return labels

data['Label'] = label_data(data)

# Drop rows with NaN labels
data = data.dropna(subset=['Label'])

# Step 2: Splitting the data
train_size = int(len(data) * 0.7)
val_size = int(len(data) * 0.2)
test_size = len(data) - train_size - val_size

train_data = data.iloc[:train_size]
val_data = data.iloc[train_size:train_size + val_size]
test_data = data.iloc[train_size + val_size:]

# Step 3: Normalizing the data (without data leakage)
scaler = StandardScaler()
features = ['Open', 'High', 'Close', 'Volume', 'RSI', 'MA']

train_data.loc[:, features] = scaler.fit_transform(train_data[features])
val_data.loc[:, features] = scaler.transform(val_data[features])
test_data.loc[:, features] = scaler.transform(test_data[features])

# Ensure there are no NaN values
assert not train_data[features].isnull().values.any(), "Train data contains NaN values"
assert not val_data[features].isnull().values.any(), "Validation data contains NaN values"
assert not test_data[features].isnull().values.any(), "Test data contains NaN values"

# Step 4: Creating sequences
def create_sequences(data, time_steps=4):
    X, y = [], []
    for i in range(len(data) - time_steps + 1):
        X.append(data.iloc[i:(i + time_steps), :].drop(columns=['Datetime', 'Label']).values)
        y.append(data.iloc[i + time_steps - 1]['Label'])
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_data)
X_val, y_val = create_sequences(val_data)
X_test, y_test = create_sequences(test_data)

# Ensure all data is of type float
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)
X_val = X_val.astype(np.float32)
y_val = y_val.astype(np.float32)
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.float32)

# Check the shapes and types of the data
print(f"X_train shape: {X_train.shape}, dtype: {X_train.dtype}")
print(f"y_train shape: {y_train.shape}, dtype: {y_train.dtype}")
print(f"X_val shape: {X_val.shape}, dtype: {X_val.dtype}")
print(f"y_val shape: {y_val.shape}, dtype: {y_val.dtype}")
print(f"X_test shape: {X_test.shape}, dtype: {X_test.dtype}")
print(f"y_test shape: {y_test.shape}, dtype: {y_test.dtype}")

# Step 5: Building the CNN model
model = Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1], X_train.shape[2])),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 6: Training the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

# Step 7: Evaluating the model
def evaluate_model(model, X, y):
    y_pred = model.predict(X).round()
    auroc = roc_auc_score(y, y_pred)
    auprc = average_precision_score(y, y_pred)
    accuracy = accuracy_score(y, y_pred)
    precision = precision_score(y, y_pred)
    recall = recall_score(y, y_pred)
    f1 = f1_score(y, y_pred)
    return auroc, auprc, accuracy, precision, recall, f1

# Evaluation on training-validation set
train_val_auroc, train_val_auprc, train_val_accuracy, train_val_precision, train_val_recall, train_val_f1 = evaluate_model(model, X_train, y_train)
val_auroc, val_auprc, val_accuracy, val_precision, val_recall, val_f1 = evaluate_model(model, X_val, y_val)

# Evaluation on test set
test_auroc, test_auprc, test_accuracy, test_precision, test_recall, test_f1 = evaluate_model(model, X_test, y_test)

print("Training-Validation Metrics:")
print(f"AUROC: {train_val_auroc}")
print(f"AUPRC: {train_val_auprc}")
print(f"Accuracy: {train_val_accuracy}")
print(f"Precision: {train_val_precision}")
print(f"Recall: {train_val_recall}")
print(f"F1 Score: {train_val_f1}")

print("\nValidation Metrics:")
print(f"AUROC: {val_auroc}")
print(f"AUPRC: {val_auprc}")
print(f"Accuracy: {val_accuracy}")
print(f"Precision: {val_precision}")
print(f"Recall: {val_recall}")
print(f"F1 Score: {val_f1}")

print("\nTest Metrics:")
print(f"AUROC: {test_auroc}")
print(f"AUPRC: {test_auprc}")
print(f"Accuracy: {test_accuracy}")
print(f"Precision: {test_precision}")
print(f"Recall: {test_recall}")
print(f"F1 Score: {test_f1}")

X_train shape: (12244, 4, 6), dtype: float32
y_train shape: (12244,), dtype: float32
X_val shape: (3496, 4, 6), dtype: float32
y_val shape: (3496,), dtype: float32
X_test shape: (1747, 4, 6), dtype: float32
y_test shape: (1747,), dtype: float32
Epoch 1/50
383/383 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5231 - loss: 0.6944 - val_accuracy: 0.5478 - val_loss: 0.6916
Epoch 2/50
383/383 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5431 - loss: 0.6859 - val_accuracy: 0.5475 - val_loss: 0.6845
Epoch 3/50
383/383 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5496 - loss: 0.6857 - val_accuracy: 0.5638 - val_loss: 0.6821
Epoch 4/50
383/383 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5587 - loss: 0.6801 - val_accuracy: 0.5535 - val_loss: 0.6833
Epoch 5/50
383/383 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5569 - loss: 0.6796 - val_accuracy: 0.5589 - val_loss: 0.6789
Epoch 6/50
383/383 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5619 - loss: 0.6812 - val_accuracy: 0.5549 - val_

In [4]:
# CNN-LSTM MODEL 

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten

# Load the time series data
data = pd.read_csv('NMC.csv')

# Ensure the data has the necessary columns: 'Open', 'High', 'Close', 'Volume', 'RSI', 'MA'
assert set(['Open', 'High', 'Close', 'Volume', 'RSI', 'MA']).issubset(data.columns), "Missing necessary columns in the data"

# Step 1: Labeling the data
def label_data(df, interval=4):
    labels = []
    for i in range(0, len(df) - interval + 1):
        if df['Open'][i] < df['Close'][i + interval - 1]:
            labels.append(1)
        else:
            labels.append(0)
    labels.extend([np.nan] * (interval - 1))
    return labels

data['Label'] = label_data(data)

# Drop rows with NaN labels
data = data.dropna(subset=['Label'])

# Step 2: Splitting the data
train_size = int(len(data) * 0.7)
val_size = int(len(data) * 0.2)
test_size = len(data) - train_size - val_size

train_data = data.iloc[:train_size]
val_data = data.iloc[train_size:train_size + val_size]
test_data = data.iloc[train_size + val_size:]

# Step 3: Normalizing the data (without data leakage)
scaler = StandardScaler()
features = ['Open', 'High', 'Close', 'Volume', 'RSI', 'MA']

train_data.loc[:, features] = scaler.fit_transform(train_data[features])
val_data.loc[:, features] = scaler.transform(val_data[features])
test_data.loc[:, features] = scaler.transform(test_data[features])

# Ensure there are no NaN values
assert not train_data[features].isnull().values.any(), "Train data contains NaN values"
assert not val_data[features].isnull().values.any(), "Validation data contains NaN values"
assert not test_data[features].isnull().values.any(), "Test data contains NaN values"

# Step 4: Creating sequences
def create_sequences(data, time_steps=4):
    X, y = [], []
    for i in range(len(data) - time_steps + 1):
        X.append(data.iloc[i:(i + time_steps), :].drop(columns=['Datetime', 'Label']).values)
        y.append(data.iloc[i + time_steps - 1]['Label'])
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_data)
X_val, y_val = create_sequences(val_data)
X_test, y_test = create_sequences(test_data)

# Ensure all data is of type float
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)
X_val = X_val.astype(np.float32)
y_val = y_val.astype(np.float32)
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.float32)

# Check the shapes and types of the data
print(f"X_train shape: {X_train.shape}, dtype: {X_train.dtype}")
print(f"y_train shape: {y_train.shape}, dtype: {y_train.dtype}")
print(f"X_val shape: {X_val.shape}, dtype: {X_val.dtype}")
print(f"y_val shape: {y_val.shape}, dtype: {y_val.dtype}")
print(f"X_test shape: {X_test.shape}, dtype: {X_test.dtype}")
print(f"y_test shape: {y_test.shape}, dtype: {y_test.dtype}")

# Step 5: Building the CNN-LSTM model
model = Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1], X_train.shape[2])),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    LSTM(128),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 6: Training the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

# Step 7: Evaluating the model
def evaluate_model(model, X, y):
    y_pred = model.predict(X).round()
    auroc = roc_auc_score(y, y_pred)
    auprc = average_precision_score(y, y_pred)
    accuracy = accuracy_score(y, y_pred)
    precision = precision_score(y, y_pred)
    recall = recall_score(y, y_pred)
    f1 = f1_score(y, y_pred)
    return auroc, auprc, accuracy, precision, recall, f1

# Evaluation on training-validation set
train_val_auroc, train_val_auprc, train_val_accuracy, train_val_precision, train_val_recall, train_val_f1 = evaluate_model(model, X_train, y_train)
val_auroc, val_auprc, val_accuracy, val_precision, val_recall, val_f1 = evaluate_model(model, X_val, y_val)

# Evaluation on test set
test_auroc, test_auprc, test_accuracy, test_precision, test_recall, test_f1 = evaluate_model(model, X_test, y_test)

print("Training-Validation Metrics:")
print(f"AUROC: {train_val_auroc}")
print(f"AUPRC: {train_val_auprc}")
print(f"Accuracy: {train_val_accuracy}")
print(f"Precision: {train_val_precision}")
print(f"Recall: {train_val_recall}")
print(f"F1 Score: {train_val_f1}")

print("\nValidation Metrics:")
print(f"AUROC: {val_auroc}")
print(f"AUPRC: {val_auprc}")
print(f"Accuracy: {val_accuracy}")
print(f"Precision: {val_precision}")
print(f"Recall: {val_recall}")
print(f"F1 Score: {val_f1}")

print("\nTest Metrics:")
print(f"AUROC: {test_auroc}")
print(f"AUPRC: {test_auprc}")
print(f"Accuracy: {test_accuracy}")
print(f"Precision: {test_precision}")
print(f"Recall: {test_recall}")
print(f"F1 Score: {test_f1}")

X_train shape: (12244, 4, 6), dtype: float32
y_train shape: (12244,), dtype: float32
X_val shape: (3496, 4, 6), dtype: float32
y_val shape: (3496,), dtype: float32
X_test shape: (1747, 4, 6), dtype: float32
y_test shape: (1747,), dtype: float32
Epoch 1/50
383/383 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5290 - loss: 0.6920 - val_accuracy: 0.5463 - val_loss: 0.6884
Epoch 2/50
383/383 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5431 - loss: 0.6874 - val_accuracy: 0.5575 - val_loss: 0.6829
Epoch 3/50
383/383 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5502 - loss: 0.6828 - val_accuracy: 0.5566 - val_loss: 0.6814
Epoch 4/50
383/383 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5493 - loss: 0.6814 - val_accuracy: 0.5592 - val_loss: 0.6808
Epoch 5/50
383/383 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5687 - loss: 0.6781 - val_accuracy: 0.5586 - val_loss: 0.6802
Epoch 6/50
383/383 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5596 - loss: 0.6776 - val_accuracy: 0.5646 - val_

In [ ]:
# ViT Model 
# TRAIN AND VALIDATION PROCESS

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import timm
import torch.nn as nn
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, accuracy_score, precision_score, recall_score, f1_score

# Load and preprocess the split datasets
train_df = pd.read_csv('train_DOGE_images_with_labels.csv')
val_df = pd.read_csv('val_DOGE_images_with_labels.csv')

# Define the dataset class
class CryptoDataset(Dataset):
    def __init__(self, data_frame, transform=None):
        self.data_frame = data_frame
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = self.data_frame.iloc[idx]['image_path']
        image = Image.open(img_name).convert('RGB')  # Convert to RGB
        label = int(self.data_frame.iloc[idx]['label'])

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create datasets
train_dataset = CryptoDataset(data_frame=train_df, transform=transform)
val_dataset = CryptoDataset(data_frame=val_df, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)  # Ensuring no shuffle
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Load pre-trained ViT model
model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=2)
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training and validation loop
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    val_loss = 0.0
    val_correct = 0
    val_total = 0
    all_labels = []
    all_predictions = []
    all_probs = []
    model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            probs = torch.softmax(outputs, dim=1)[:, 1]
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    auroc = roc_auc_score(all_labels, all_probs)
    precision, recall, _ = precision_recall_curve(all_labels, all_probs)
    auprc = auc(recall, precision)
    precision_score_value = precision_score(all_labels, all_predictions)
    recall_score_value = recall_score(all_labels, all_predictions)
    f1_score_value = f1_score(all_labels, all_predictions)

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}, Val Loss: {val_loss/len(val_loader)}, Val Accuracy: {val_accuracy}, AUROC: {auroc}, AUPRC: {auprc}, Precision: {precision_score_value}, Recall: {recall_score_value}, F1 Score: {f1_score_value}')

# Save the model after training
torch.save(model.state_dict(), 'DOGE_trained_model_vit.pth')

# IMPORT TEST SET
import pandas as pd 

# Load the test dataset
test_df = pd.read_csv('test_DOGE_images_with_labels.csv')

print(test_df)

# TEST PROCESS
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import timm
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score, precision_score, recall_score, f1_score
import numpy as np 

# Define the dataset class
class CryptoDataset(Dataset):
    def __init__(self, data_frame, transform=None):
        self.data_frame = data_frame
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = self.data_frame.iloc[idx]['image_path']
        label = self.data_frame.iloc[idx]['label']  # Assuming 'label' is the column name
        image = Image.open(img_name).convert('RGB')  # Convert to RGB

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transformations (same as before)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create the test dataset and DataLoader
test_dataset = CryptoDataset(data_frame=test_df, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load the pre-trained model (same architecture as before)
model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=2)
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Load the trained model weights
model.load_state_dict(torch.load('DOGE_trained_model_vit.pth'))
model.eval()

# Testing loop
all_test_predictions = []
all_test_labels = []

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        all_test_predictions.extend(predicted.cpu().numpy())
        all_test_labels.extend(labels.cpu().numpy())

# Convert predictions and labels to numpy arrays
all_test_predictions = np.array(all_test_predictions)
all_test_labels = np.array(all_test_labels)

# Calculate metrics
accuracy = accuracy_score(all_test_labels, all_test_predictions)
precision = precision_score(all_test_labels, all_test_predictions, average='weighted')
recall = recall_score(all_test_labels, all_test_predictions, average='weighted')
f1 = f1_score(all_test_labels, all_test_predictions, average='weighted')
auroc = roc_auc_score(all_test_labels, all_test_predictions, average='weighted', multi_class='ovr')
auprc = average_precision_score(all_test_labels, all_test_predictions, average='weighted')

# Print metrics
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(f'AUROC: {auroc}')
print(f'AUPRC: {auprc}')

In [ ]:
# DeiT Model 

# TRAIN AND VALIDATION PROCESS

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import timm
import torch.nn as nn
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, accuracy_score, precision_score, recall_score, f1_score

# Load and preprocess the split datasets
train_df = pd.read_csv('train_DOGE_images_with_labels.csv')
val_df = pd.read_csv('val_DOGE_images_with_labels.csv')

# Define the dataset class
class CryptoDataset(Dataset):
    def __init__(self, data_frame, transform=None):
        self.data_frame = data_frame
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = self.data_frame.iloc[idx]['image_path']
        image = Image.open(img_name).convert('RGB')  # Convert to RGB
        label = int(self.data_frame.iloc[idx]['label'])

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create datasets
train_dataset = CryptoDataset(data_frame=train_df, transform=transform)
val_dataset = CryptoDataset(data_frame=val_df, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)  # Ensuring no shuffle
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Load pre-trained DeiT model
model = timm.create_model('deit_base_patch16_224', pretrained=True, num_classes=2)
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training and validation loop
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    val_loss = 0.0
    val_correct = 0
    val_total = 0
    all_labels = []
    all_predictions = []
    all_probs = []
    model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            probs = torch.softmax(outputs, dim=1)[:, 1]
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    auroc = roc_auc_score(all_labels, all_probs)
    precision, recall, _ = precision_recall_curve(all_labels, all_probs)
    auprc = auc(recall, precision)
    precision_score_value = precision_score(all_labels, all_predictions)
    recall_score_value = recall_score(all_labels, all_predictions)
    f1_score_value = f1_score(all_labels, all_predictions)

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}, Val Loss: {val_loss/len(val_loader)}, Val Accuracy: {val_accuracy}, AUROC: {auroc}, AUPRC: {auprc}, Precision: {precision_score_value}, Recall: {recall_score_value}, F1 Score: {f1_score_value}')

# Save the model after training
torch.save(model.state_dict(), 'DOGE_trained_model_deit.pth')

# IMPORT TEST SET
import pandas as pd 

# Load the test dataset
test_df = pd.read_csv('test_DOGE_images_with_labels.csv')

print(test_df)

# TEST PROCESS
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import timm
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score, precision_score, recall_score, f1_score
import numpy as np 

# Define the dataset class
class CryptoDataset(Dataset):
    def __init__(self, data_frame, transform=None):
        self.data_frame = data_frame
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = self.data_frame.iloc[idx]['image_path']
        label = self.data_frame.iloc[idx]['label']  # Assuming 'label' is the column name
        image = Image.open(img_name).convert('RGB')  # Convert to RGB

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transformations (same as before)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create the test dataset and DataLoader
test_dataset = CryptoDataset(data_frame=test_df, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load the pre-trained model (same architecture as before)
model = timm.create_model('deit_base_patch16_224', pretrained=True, num_classes=2)
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Load the trained model weights
model.load_state_dict(torch.load('DOGE_trained_model_deit.pth'))
model.eval()

# Testing loop
all_test_predictions = []
all_test_labels = []

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        all_test_predictions.extend(predicted.cpu().numpy())
        all_test_labels.extend(labels.cpu().numpy())

# Convert predictions and labels to numpy arrays
all_test_predictions = np.array(all_test_predictions)
all_test_labels = np.array(all_test_labels)

# Calculate metrics
accuracy = accuracy_score(all_test_labels, all_test_predictions)
precision = precision_score(all_test_labels, all_test_predictions, average='weighted')
recall = recall_score(all_test_labels, all_test_predictions, average='weighted')
f1 = f1_score(all_test_labels, all_test_predictions, average='weighted')
auroc = roc_auc_score(all_test_labels, all_test_predictions, average='weighted', multi_class='ovr')
auprc = average_precision_score(all_test_labels, all_test_predictions, average='weighted')

# Print metrics
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(f'AUROC: {auroc}')
print(f'AUPRC: {auprc}')

In [ ]:
# ResNet50 Model 

# TRAIN AND VALIDATION PROCESS

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import timm
import torch.nn as nn
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, accuracy_score, precision_score, recall_score, f1_score

# Load and preprocess the split datasets
train_df = pd.read_csv('train_DOGE_images_with_labels.csv')
val_df = pd.read_csv('val_DOGE_images_with_labels.csv')

# Define the dataset class
class CryptoDataset(Dataset):
    def __init__(self, data_frame, transform=None):
        self.data_frame = data_frame
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = self.data_frame.iloc[idx]['image_path']
        image = Image.open(img_name).convert('RGB')  # Convert to RGB
        label = int(self.data_frame.iloc[idx]['label'])

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create datasets
train_dataset = CryptoDataset(data_frame=train_df, transform=transform)
val_dataset = CryptoDataset(data_frame=val_df, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)  # Ensuring no shuffle
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Load pre-trained ResNet-50 model
model = timm.create_model('resnet50', pretrained=True, num_classes=2)
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training and validation loop
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    val_loss = 0.0
    val_correct = 0
    val_total = 0
    all_labels = []
    all_predictions = []
    all_probs = []
    model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            probs = torch.softmax(outputs, dim=1)[:, 1]
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    auroc = roc_auc_score(all_labels, all_probs)
    precision, recall, _ = precision_recall_curve(all_labels, all_probs)
    auprc = auc(recall, precision)
    precision_score_value = precision_score(all_labels, all_predictions)
    recall_score_value = recall_score(all_labels, all_predictions)
    f1_score_value = f1_score(all_labels, all_predictions)

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}, Val Loss: {val_loss/len(val_loader)}, Val Accuracy: {val_accuracy}, AUROC: {auroc}, AUPRC: {auprc}, Precision: {precision_score_value}, Recall: {recall_score_value}, F1 Score: {f1_score_value}')

# Save the model after training
torch.save(model.state_dict(), 'DOGE_trained_model_resnet50.pth')

# IMPORT TEST SET
import pandas as pd 

# Load the test dataset
test_df = pd.read_csv('test_DOGE_images_with_labels.csv')

print(test_df)

# TEST PROCESS
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import timm
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score, precision_score, recall_score, f1_score
import numpy as np 

# Define the dataset class
class CryptoDataset(Dataset):
    def __init__(self, data_frame, transform=None):
        self.data_frame = data_frame
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = self.data_frame.iloc[idx]['image_path']
        label = self.data_frame.iloc[idx]['label']  # Assuming 'label' is the column name
        image = Image.open(img_name).convert('RGB')  # Convert to RGB

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transformations (same as before)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create the test dataset and DataLoader
test_dataset = CryptoDataset(data_frame=test_df, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load the pre-trained model (same architecture as before)
model = timm.create_model('resnet50', pretrained=True, num_classes=2)
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Load the trained model weights
model.load_state_dict(torch.load('DOGE_trained_model_resnet50.pth'))
model.eval()

# Testing loop
all_test_predictions = []
all_test_labels = []

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        all_test_predictions.extend(predicted.cpu().numpy())
        all_test_labels.extend(labels.cpu().numpy())

# Convert predictions and labels to numpy arrays
all_test_predictions = np.array(all_test_predictions)
all_test_labels = np.array(all_test_labels)

# Calculate metrics
accuracy = accuracy_score(all_test_labels, all_test_predictions)
precision = precision_score(all_test_labels, all_test_predictions, average='weighted')
recall = recall_score(all_test_labels, all_test_predictions, average='weighted')
f1 = f1_score(all_test_labels, all_test_predictions, average='weighted')
auroc = roc_auc_score(all_test_labels, all_test_predictions, average='weighted', multi_class='ovr')
auprc = average_precision_score(all_test_labels, all_test_predictions, average='weighted')

# Print metrics
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(f'AUROC: {auroc}')
print(f'AUPRC: {auprc}')